In [1]:
import sys
import os 
import cv2
import glob
import random
import numpy as np
from util.rotation_operator import rotate_nodes_dict, rotate
from data_process.gen_refine_targets_helper import construct_sub_graph, gen_node_cls_reg_targets, gen_adj_conn_targets

In [2]:

# A list of maps adding as negative thrust fault lines for fault line training
#'AK_HinesCreek','CO_HorseMtn','DC_Wash_West','ID_basement','ID_LowerValley','VA_Lahore_bm','CA_BartlettSprings'
#'AZ_PeachSprings','AK_Christian'



In [3]:
data_dir = '/data/weiweidu/data/training_png_shp'
map_name = 'AK_HinesCreek'

pts_fname = '%s_thrust_fault_line.txt'%(map_name)
WIN_SIZE = 512
gsize = 32
num_nodes = 200
map_name = '%s.png'%(map_name)
map_path = os.path.join(data_dir, map_name)

training_folder_name = 'fault_line_samples_img512_grid32_color_aug'

In [4]:
pts = np.loadtxt(os.path.join(data_dir, pts_fname), dtype=np.int32, delimiter=",")
np.random.shuffle(pts)
print('{} pts in the txt file.'.format(pts.shape))

(192, 2) pts in the txt file.


In [5]:
n_samples = len(glob.glob(os.path.join(os.path.join(data_dir, training_folder_name), '*.npz')))

print('{} samples in the folder.'.format(n_samples))

10541 samples in the folder.


In [6]:
# temp_folder = 'new_lineament_line_samples'

# if not os.path.exists(os.path.join(data_dir, temp_folder)):
#     os.makedirs(os.path.join(data_dir, temp_folder))

In [7]:
map_img = cv2.imread(map_path)
print('map image shape is {}'.format(map_img.shape))

map image shape is (16215, 23813, 3)


In [8]:
for i, pt in enumerate(pts[:100]):
    sub_nodes_dict, sub_edges_list = {}, []

    for r_angle in random.sample([90, 180, 270, 360], 1):
        rot_sub_nodes_dict = {}
        node_cat_targets, node_reg_targets, nodes_in_grids, rot_sub_nodes_dict, sub_edges_list \
                = gen_node_cls_reg_targets(rot_sub_nodes_dict, sub_edges_list, gsize=gsize, img_size=WIN_SIZE)

        node_inputs, adj_targets, conn_targets, nodes_mask = \
            gen_adj_conn_targets(nodes_in_grids, rot_sub_nodes_dict, sub_edges_list, gsize=gsize, img_size=WIN_SIZE, n_nodes=num_nodes)

        x_img = map_img[pt[0]-WIN_SIZE//2:pt[0]+WIN_SIZE//2, pt[1]-WIN_SIZE//2:pt[1]+WIN_SIZE//2]
        rot_x_img = rotate(x_img, angle=r_angle)

        # write sample to disk
        file_name = os.path.join(data_dir, f'{training_folder_name}', f'{i+n_samples}.npz')
        data = {
            'x_img': rot_x_img,
            'node_cat_targets': node_cat_targets,
            'node_reg_targets': node_reg_targets,
            'node_inputs': node_inputs,
            'adj_targets': adj_targets,
            'conn_targets': conn_targets,
            'nodes_mask': nodes_mask}
        np.savez(file_name, **data)

In [9]:
# for root, dirs, files in os.walk("/data/weiweidu/data/training_png_shp/fault_line_samples_img512_grid32", topdown=False):
#     for name in files:
#         index = int(name.split('.')[0])
#         if index > 9508:
#             os.remove(os.path.join(root, name))